# Expresiones en Polars: Introducción y Ejemplos

Las **expresiones** en Polars son el núcleo de la manipulación eficiente de datos. Por sí solas, las expresiones son *perezosas* (lazy): no hacen nada hasta que se aplican sobre un `DataFrame` o `LazyFrame` mediante métodos específicos.

En este notebook, exploraremos cómo las expresiones se utilizan en la práctica a través ejemplos claros y reproducibles. Veremos cómo aplicar expresiones para:

- **Seleccionar columnas** con `df.select()`
- **Crear nuevas columnas** con `df.with_columns()`
- **Filtrar filas** con `df.filter()`
- **Agrupar y agregar** con `df.group_by()`
- **Ordenar filas** con `df.sort()`

> **Nota:** Aunque cada método tiene su importancia, lo fundamental es entender cómo las expresiones definen la lógica de transformación y análisis de los datos. Cada método será tratado en detalle en capítulos posteriores.

---

Este notebook sigue las buenas prácticas del proyecto: los ejemplos son reproducibles, los datos son ligeros y se utiliza Polars para la manipulación y Plotnine para la visualización. ¡Comencemos!

In [1]:
import polars as pl 

In [2]:
fruit = pl.read_csv("data/fruit.csv")
fruit

name,weight,color,is_round,origin
str,i64,str,bool,str
"""Avocado""",200,"""green""",false,"""South America"""
"""Banana""",120,"""yellow""",false,"""Asia"""
"""Blueberry""",1,"""blue""",false,"""North America"""
"""Cantaloupe""",2500,"""orange""",true,"""Africa"""
"""Cranberry""",2,"""red""",false,"""North America"""
"""Elderberry""",1,"""black""",false,"""Europe"""
"""Orange""",130,"""orange""",true,"""Asia"""
"""Papaya""",1000,"""orange""",false,"""South America"""
"""Peach""",150,"""orange""",true,"""Asia"""


In [3]:
fruit.select(
    pl.col("name"),
    pl.col("^.*or.*$"),
    pl.col("weight") / 1000,
    "is_round"
)

name,color,origin,weight,is_round
str,str,str,f64,bool
"""Avocado""","""green""","""South America""",0.2,false
"""Banana""","""yellow""","""Asia""",0.12,false
"""Blueberry""","""blue""","""North America""",0.001,false
"""Cantaloupe""","""orange""","""Africa""",2.5,true
"""Cranberry""","""red""","""North America""",0.002,false
"""Elderberry""","""black""","""Europe""",0.001,false
"""Orange""","""orange""","""Asia""",0.13,true
"""Papaya""","""orange""","""South America""",1.0,false
"""Peach""","""orange""","""Asia""",0.15,true


### Creando nuevas columnas con expresiones

In [5]:
fruit.with_columns(
    pl.lit(True).alias("is_fruit"),
    is_berry=pl.col("name").str.ends_with("berry")
)

name,weight,color,is_round,origin,is_fruit,is_berry
str,i64,str,bool,str,bool,bool
"""Avocado""",200,"""green""",false,"""South America""",true,false
"""Banana""",120,"""yellow""",false,"""Asia""",true,false
"""Blueberry""",1,"""blue""",false,"""North America""",true,true
"""Cantaloupe""",2500,"""orange""",true,"""Africa""",true,false
"""Cranberry""",2,"""red""",false,"""North America""",true,true
"""Elderberry""",1,"""black""",false,"""Europe""",true,true
"""Orange""",130,"""orange""",true,"""Asia""",true,false
"""Papaya""",1000,"""orange""",false,"""South America""",true,false
"""Peach""",150,"""orange""",true,"""Asia""",true,false


### Nombrado de columnas en Polars

Al crear nuevas columnas con expresiones en Polars, existen dos formas principales de asignarles nombre:

- **Usando `Expr.alias()`**: El método `.alias()` asigna un nombre directamente a la expresión, que se mantiene dondequiera que se use esa expresión.
- **Usando sintaxis de palabra clave**: Al pasar la expresión como argumento nombrado (`nuevo_nombre=expresión`), el nombre es local al contexto y tiene prioridad sobre `.alias()` si ambos se usan.

**Consideraciones:**
- Si se usa una palabra clave, todos los argumentos siguientes deben ser palabras clave.
- El nombre debe ser válido en Python: no puede comenzar con un número, contener caracteres especiales ni ser una palabra reservada.

**Recomendación:**  
Se prefiere la sintaxis de palabra clave por su claridad y brevedad.

## Filtrando filas con expresiones

In [8]:
fruit.filter(
    (pl.col("weight") > 1000)
    & pl.col("is_round")
)

name,weight,color,is_round,origin
str,i64,str,bool,str
"""Cantaloupe""",2500,"""orange""",true,"""Africa"""
"""Watermelon""",5000,"""green""",true,"""Africa"""


## Agregación con expresiones

In [19]:
fruit.group_by(
    pl.col("origin").str.split(" ").list.last()).agg(
        pl.len(),
        average_weight=pl.col("weight").mean()
    ).sort("average_weight", descending=True)

origin,len,average_weight
str,u32,f64
"""Africa""",2,3750.0
"""America""",4,300.75
"""Asia""",3,133.333333
"""Europe""",1,1.0


## Orden con expresiones

In [13]:
fruit.sort(
    pl.col("name").str.len_bytes(),
    descending=True
)

name,weight,color,is_round,origin
str,i64,str,bool,str
"""Cantaloupe""",2500,"""orange""",true,"""Africa"""
"""Elderberry""",1,"""black""",false,"""Europe"""
"""Watermelon""",5000,"""green""",true,"""Africa"""
"""Blueberry""",1,"""blue""",false,"""North America"""
"""Cranberry""",2,"""red""",false,"""North America"""
"""Avocado""",200,"""green""",false,"""South America"""
"""Banana""",120,"""yellow""",false,"""Asia"""
"""Orange""",130,"""orange""",true,"""Asia"""
"""Papaya""",1000,"""orange""",false,"""South America"""


In [20]:
(pl.DataFrame({"a": [1, 2, 3], "b":  [0.4, 0.5, 0.6]}).with_columns(
    pl.all().mul(10).name.suffix("_times_10")
    )
)

a,b,a_times_10,b_times_10
i64,f64,i64,f64
1,0.4,10,4.0
2,0.5,20,5.0
3,0.6,30,6.0


In [21]:
pl.all().mul(10).name.suffix("_times_10").meta.has_multiple_outputs()

True